In [1]:
!pip install transformers
import pandas as pd
import nltk
from tqdm.notebook import tqdm
tqdm.pandas()

# Load Readability data

In [2]:
# For Google Colab
#from google.colab import drive
#drive.mount('/content/drive')
#df_readability_kickstarter_small = pd.read_csv('./kickstarter_readability_small.csv')
#df_readability_kickstarter = pd.read_csv('../datasets/kickstarter_cleaned/kickstarter_cleaned_topicmodelling.csv')
df_progess = pd.read_csv('./progress.csv')

In [11]:
df_progess['bert_score'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 356108 entries, 0 to 356107
Series name: bert_score
Non-Null Count   Dtype  
--------------   -----  
356108 non-null  float64
dtypes: float64(1)
memory usage: 2.7 MB


In [ ]:
df_progess.iloc[335000:].head(10) # 220000 # 260000
#print(len(df_progess.iloc[90000:].index))

## Calculate BERT PPL Score

In [6]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

model_name = 'bert-base-uncased'
model = AutoModelForMaskedLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.cuda().half()
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def score(model, tokenizer, sentence):
    #print(torch.cuda.memory_allocated() // 1024 ** 2)
    with torch.no_grad():
        tensor_input = tokenizer.encode(sentence, return_tensors='pt')
        
        # Check if the tokenized input exceeds GPU RAM limit
        if tensor_input.size(-1) > 256:
            # Truncate the sentence to a maximum length
            tensor_input = tensor_input[:, :256]
            
        #print(tensor_input.size())
        repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
        mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
        masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
        #print(masked_input.size())
        labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    
        # Send masked_input and labels to the GPU
        masked_input = masked_input.cuda()
        labels = labels.cuda()
        
        #print(torch.cuda.memory_allocated() // 1024 ** 2)
        
        with torch.inference_mode():
            loss = model(masked_input, labels=labels).loss
    
        #print(torch.cuda.memory_allocated() // 1024 ** 2)
        
        bert_score = np.exp(loss.item())
    
        # Delete intermediate tensors
        masked_input = None
        labels = None
        del tensor_input, repeat_input, mask, masked_input, labels, loss
        
        #print(torch.cuda.memory_allocated() // 1024 ** 2)
        
        return bert_score

In [ ]:
print(model)

In [ ]:
score(model, tokenizer, "How do you like me?")

In [8]:
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

#def score_corpus(corpus):
    #sentences = sent_tokenize(corpus)
    #batch_size = 1  # Adjust batch size according to your GPU memory
    
    #scores = []
    #for i in range(0, len(sentences), batch_size):
    #    batch_sentences = sentences[i:i + batch_size]
        
    #    with torch.inference_mode():
    #        batch_scores = [score(model, tokenizer, sentence) for sentence in batch_sentences]
        
    #    batch_sentences = None
    #    del batch_sentences
        
    #    scores.extend(batch_scores)
        
        # Clear GPU memory after each batch
    #    torch.cuda.empty_cache()
    
    # mean = geo_mean(scores)
    
    # Delete intermediate Tensors
    # del scores
    
    #return mean
    
def score_corpus(corpus):
    sentences = sent_tokenize(corpus)
    scores = []
    for sentence in sentences:
        #print(sentence)
        bert_score = score(model, tokenizer, sentence)
        #print(bert_score)
        scores.append(bert_score)
        
        del bert_score
        del sentence
        #torch.cuda.empty_cache()
    
    #print(scores)
    mean = geo_mean(scores)
    #print(mean)
    
    del scores
    
    return mean


#scores = [score(model, tokenizer, sentence) for sentence in sentences]
    
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
def geo_mean(iterable):  
    a = torch.tensor(iterable, device='cuda')
    # Check for NaN values in the tensor and filter them out
    a = a[~torch.isnan(a)]  
    mean = torch.prod(a).pow(1.0 / len(a))
    del a
    return mean.item()

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [10]:
# Define the interval for saving the DataFrame
save_interval = 5000  # Adjust as needed

# Initialize a counter for tracking the number of rows processed
row_counter = 0

# Process the DataFrame and update 'bert_score' column
for index, row in tqdm(df_progess.iloc[335000:].iterrows()):
    corpus = row['project_description']
    bert_score = score_corpus(corpus)
    
    # Update the 'bert_score' column in the original DataFrame
    df_progess.at[index, 'bert_score'] = bert_score
     
    # Increment the row counter
    row_counter += 1
    
    # Check if it's time to save the DataFrame
    if row_counter % save_interval == 0:
        # Save the DataFrame to a CSV file (e.g., "progress.csv")
        df_progess.to_csv('./progress.csv')

# Save the final result to a separate CSV file
df_progess.to_csv('./kickstarter_cleaned_BERTPPL.csv')


0it [00:00, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (635 > 512). Running this sequence through the model will result in indexing errors


# Plots

In [3]:
import pandas as pd

df_bert = pd.read_csv('../kickstarter_cleaned_BERTPPL_Readability.csv')

In [6]:
df_bert['date'] = pd.to_datetime(df_bert['project_launched_at'])
df_bert = df_bert.groupby('date')['bert_score'].mean().reset_index()
df_bert = df_bert.sort_values('date')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
# Apply LOESS smoothing
lowess = sm.nonparametric.lowess(df_bert['bert_score'], df_bert.index, frac=0.1)

plt.figure(figsize=(10, 6))
plt.plot(df_bert['date'], lowess[:, 1])
plt.xlabel('Date')
plt.ylabel('Smoothed BERT Score')
plt.title('LOESS BERT Score Over Time')
plt.show()